A sample analysis workflow for R, starting with count data.  
Timothy J. Eisen  
April 7, 2022

In [2]:
#Imports
library(tidyverse)
source("ggplot_theme.R") #Tim's themes
source("r_helpers.R") #Some multiuse code
library(readxl)
library(ggpubr)
library(ggrepel)
library(corrplot)
library(cowplot)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



log_ticks usage:
labels = log_ticks(start,end)[[1]]
breaks = log_ticks(start,end)[[2]] 
row order: E D K R H P Q N T S A G M C V I L Y F W X 



Attaching package: ‘ggpubr’


The following object is masked from ‘package:cowplot’:

    get_legend




Loading tidyverse, cowplot, ggpubr, and gtools.
Functions: plot_corr_grid(df)
Assume all columns are numeric
Pearson or spearman?


corrplot 0.84 loaded



In [3]:
#parameters
READ_CUTOFF = 50
PSEUDOCOUNT = 1
BACKGROUND = 0 #How much background should be subtracted? 0.22 is the ratio of masses after the gel extraction. 

In [4]:
select <- dplyr::select #Don't overload the select operator from tidy, MASS package

In [5]:
#This file is a tab-delimited file that describes the variant name, position, and codon information. 
#I make it for each library that I prep, let me know if you want help constructing it. 
VarTable <- read_tsv("BTK_activation_loop_20210924_SeqNames.txt") %>%
    unite(col = aapos, wt_aa, pos, remove = FALSE) %>%
    unite(col = var_codon_aa, var_aa, var_codon, remove = FALSE) %>%
    print(width = Inf)

Parsed with column specification:
cols(
  seq_name = col_character(),
  wt_codon = col_character(),
  pos = col_double(),
  var_codon = col_character(),
  wt_aa = col_character(),
  var_aa = col_character()
)



# A tibble: 1,155 x 8
   seq_name               wt_codon aapos   pos var_codon_aa var_codon wt_aa
   <chr>                  <chr>    <chr> <dbl> <chr>        <chr>     <chr>
 1 TJE_WT_activation_loop WT       wt_0      0 wt_wt        wt        wt   
 2 TJE_TCT538TGA_S538X    TCT      S_538   538 X_TGA        TGA       S    
 3 TJE_TCT538TAA_S538X    TCT      S_538   538 X_TAA        TAA       S    
 4 TJE_TCT538GCC_S538A    TCT      S_538   538 A_GCC        GCC       S    
 5 TJE_TCT538GCT_S538A    TCT      S_538   538 A_GCT        GCT       S    
 6 TJE_TCT538TGC_S538C    TCT      S_538   538 C_TGC        TGC       S    
 7 TJE_TCT538TGT_S538C    TCT      S_538   538 C_TGT        TGT       S    
 8 TJE_TCT538GAC_S538D    TCT      S_538   538 D_GAC        GAC       S    
 9 TJE_TCT538GAT_S538D    TCT      S_538   538 D_GAT        GAT       S    
10 TJE_TCT538GAG_S538E    TCT      S_538   538 E_GAG        GAG       S    
   var_aa
   <chr> 
 1 wt    
 2 X     
 3 X     
 4 A     
 5 A  

In [6]:
#metadata about the count files
dataset_annotations <- read_tsv("annotations_activation_loop.txt") %>% print()

Parsed with column specification:
cols(
  file = col_character(),
  data_type = col_character(),
  rep = col_double()
)



# A tibble: 8 x 3
  file                                                   data_type   rep
  <chr>                                                  <chr>     <dbl>
1 ../Analysis20220304/grep_quant_twist3/s55_grep_out.txt input         1
2 ../Analysis20220304/grep_quant_twist3/s63_grep_out.txt sort          1
3 ../Analysis20220304/grep_quant_twist3/s71_grep_out.txt input         2
4 ../Analysis20220304/grep_quant_twist3/s56_grep_out.txt sort          2
5 ../Analysis20220304/grep_quant_twist3/s64_grep_out.txt input         3
6 ../Analysis20220304/grep_quant_twist3/s72_grep_out.txt sort          3
7 ../Analysis20220304/grep_quant_twist3/s49_grep_out.txt input         4
8 ../Analysis20220304/grep_quant_twist3/s57_grep_out.txt sort          4


The data format is a basic kallisto counting output, which I now use for everything even when I'm not using kallisto. I'll include a sample below.  
The only required fields are "target_id" and "est_counts"

In [3]:
#an example data file
read_tsv('../Analysis20220304/grep_quant_twist3/s55_grep_out.txt') %>%
    print(width = Inf)

Parsed with column specification:
cols(
  target_id = col_character(),
  sequence = col_character(),
  est_counts = col_double()
)



# A tibble: 1,155 x 3
   target_id             
   <chr>                 
 1 TJE_WT_activation_loop
 2 TJE_TCT538TGA_S538X   
 3 TJE_TCT538TAA_S538X   
 4 TJE_TCT538GCC_S538A   
 5 TJE_TCT538GCT_S538A   
 6 TJE_TCT538TGC_S538C   
 7 TJE_TCT538TGT_S538C   
 8 TJE_TCT538GAC_S538D   
 9 TJE_TCT538GAT_S538D   
10 TJE_TCT538GAG_S538E   
   sequence                                                                     
   <chr>                                                                        
 1 ACGATCAAGGAGTTGTTAAAGTATCTGATTTCGGCCTGTCCAGGTATGTCCTGGATGATGAATACACAAGCTCAGT…
 2 ACGATCAAGGAGTTGTTAAAGTATGAGATTTCGGCCTGTCCAGGTATGTCCTGGATGATGAATACACAAGCTCAGT…
 3 ACGATCAAGGAGTTGTTAAAGTATAAGATTTCGGCCTGTCCAGGTATGTCCTGGATGATGAATACACAAGCTCAGT…
 4 ACGATCAAGGAGTTGTTAAAGTAGCCGATTTCGGCCTGTCCAGGTATGTCCTGGATGATGAATACACAAGCTCAGT…
 5 ACGATCAAGGAGTTGTTAAAGTAGCTGATTTCGGCCTGTCCAGGTATGTCCTGGATGATGAATACACAAGCTCAGT…
 6 ACGATCAAGGAGTTGTTAAAGTATGCGATTTCGGCCTGTCCAGGTATGTCCTGGATGATGAATACACAAGCTCAGT…
 7 ACGATCAAGGAGTTG

In [8]:
#import the data
all_data_list <- lapply(seq(nrow(dataset_annotations)), function(x){
    file = dataset_annotations[x,]$file
    data_type = dataset_annotations[x,]$data_type
    rep = dataset_annotations[x,]$rep
    data_tib = read_tsv(file) %>% select(seq_name = target_id, est_counts) %>%
        mutate(data = data_type, rep = rep)
    return(data_tib)})

mAll <- bind_rows(all_data_list) %>% pivot_wider(names_from = data, values_from = est_counts) %>% print()
write_tsv(mAll, path = 'processed_files/CountsCompiled_20220407.txt')

Parsed with column specification:
cols(
  target_id = col_character(),
  sequence = col_character(),
  est_counts = col_double()
)

Parsed with column specification:
cols(
  target_id = col_character(),
  sequence = col_character(),
  est_counts = col_double()
)

Parsed with column specification:
cols(
  target_id = col_character(),
  sequence = col_character(),
  est_counts = col_double()
)

Parsed with column specification:
cols(
  target_id = col_character(),
  sequence = col_character(),
  est_counts = col_double()
)

Parsed with column specification:
cols(
  target_id = col_character(),
  sequence = col_character(),
  est_counts = col_double()
)

Parsed with column specification:
cols(
  target_id = col_character(),
  sequence = col_character(),
  est_counts = col_double()
)

Parsed with column specification:
cols(
  target_id = col_character(),
  sequence = col_character(),
  est_counts = col_double()
)

Parsed with column specification:
cols(
  target_id = col_character(),
  seq

# A tibble: 4,620 x 4
   seq_name                 rep input   sort
   <chr>                  <dbl> <dbl>  <dbl>
 1 TJE_WT_activation_loop     1 45213 131675
 2 TJE_TCT538TGA_S538X        1    24     38
 3 TJE_TCT538TAA_S538X        1    36     34
 4 TJE_TCT538GCC_S538A        1   159    215
 5 TJE_TCT538GCT_S538A        1   105    283
 6 TJE_TCT538TGC_S538C        1    58    218
 7 TJE_TCT538TGT_S538C        1     5    244
 8 TJE_TCT538GAC_S538D        1   142    172
 9 TJE_TCT538GAT_S538D        1    67     83
10 TJE_TCT538GAG_S538E        1    19    101
# … with 4,610 more rows


In [56]:
#Normalization of the data.
mNorm <- mAll %>% 
    mutate(
            input = input + PSEUDOCOUNT, #adding pseudocounts
            sort  = sort + PSEUDOCOUNT,
            sort_norm = log10(sort / input), #enrichments
            sort_norm = ifelse(input < READ_CUTOFF, NA, sort_norm)) %>% #cutoffs
        left_join(VarTable, by = 'seq_name') %>%
        print(width = Inf) %>%
        mutate( #which are the wt?
            wt_variant = if_else(wt_aa == var_aa | str_detect(seq_name, "_WT_"), TRUE, FALSE)) %>%
        group_by(wt_variant, rep) %>% #wt averaging
        mutate( #make the wt seq an average of all the WT seqs
            sort_norm = ifelse(seq_name == 'TJE_WT_activation_loop', mean(sort_norm, na.rm = TRUE), sort_norm)) %>%
        ungroup() %>%
        group_by(rep) %>% #this next block normalizes everything to the WT
        mutate(sort_norm = sort_norm - sort_norm[seq_name == 'TJE_WT_activation_loop']) %>% 
        ungroup() %>%
        print(width = Inf)
write_tsv(mNorm, path = 'processed_files/NormalizedData_20220407.txt')

# A tibble: 4,620 x 12
   seq_name                 rep input   sort sort_norm wt_codon aapos   pos
   <chr>                  <dbl> <dbl>  <dbl>     <dbl> <chr>    <chr> <dbl>
 1 TJE_WT_activation_loop     1 45214 131676    0.464  WT       wt_0      0
 2 TJE_TCT538TGA_S538X        1    25     39   NA      TCT      S_538   538
 3 TJE_TCT538TAA_S538X        1    37     35   NA      TCT      S_538   538
 4 TJE_TCT538GCC_S538A        1   160    216    0.130  TCT      S_538   538
 5 TJE_TCT538GCT_S538A        1   106    284    0.428  TCT      S_538   538
 6 TJE_TCT538TGC_S538C        1    59    219    0.570  TCT      S_538   538
 7 TJE_TCT538TGT_S538C        1     6    245   NA      TCT      S_538   538
 8 TJE_TCT538GAC_S538D        1   143    173    0.0827 TCT      S_538   538
 9 TJE_TCT538GAT_S538D        1    68     84    0.0918 TCT      S_538   538
10 TJE_TCT538GAG_S538E        1    20    102   NA      TCT      S_538   538
   var_codon_aa var_codon wt_aa var_aa
   <chr>        <chr>     

In [19]:
#fitness plots, histogram
p <- ggplot(mNorm, aes(x = sort_norm)) +
        geom_histogram(bins = 100) + 
        scale_y_continuous(expand = c(0,0)) + 
        geom_vline(xintercept = 0, linetype = 'dashed', color = 'grey') +
        facet_wrap(~as.character(rep)) +
        theme_tim_label()
ggsave(plot = p, file = 'plots/FitnessHistograms.pdf')

Saving 7 x 7 in image

Warning message:
“Removed 801 rows containing non-finite values (stat_bin).”


In [45]:
CountsByAA <- mNorm %>%
    filter(!wt_variant) %>% #for this analysis, remove the wt
    group_by(var_codon, rep) %>%
    summarise(sum_input = sum(input, na.rm = TRUE)) %>%
    ungroup() %>%
    mutate(Type = ifelse(var_codon %in% c("TAA", "TGA", "TAG"), "Nonsense", "Missense"))

pCountsByAA <- ggplot(CountsByAA, aes(x = sum_input, fill = Type)) +
    geom_histogram(bins = 10) +
    scale_x_continuous(expand = c(0,0)) +
    scale_y_continuous(expand = c(0,0)) +
    facet_wrap(~(as.character(rep)), scales = 'free_x') +
    theme_tim_label()

ggsave(plot = pCountsByAA, file = 'plots/pCountsByAA.pdf', width = 6, height = 4)

`summarise()` regrouping output by 'var_codon' (override with `.groups` argument)



In [49]:
#are there positions that are overrepresented in the input data?
CountsByPos <- mNorm %>%
    filter(!wt_variant) %>% #for this analysis, remove the wt variants
    group_by(var_codon_aa, rep, pos) %>%
    summarise(sum_input = sum(input, na.rm = TRUE)) %>%
    ungroup() %>%
    print()

pCountsByPos <- ggplot(CountsByPos, aes(x = pos, y = sum_input, fill = var_codon_aa)) +
    scale_x_continuous(expand = c(0,0)) +
    scale_y_continuous(expand = c(0,0)) +
    geom_col() +
    ggtitle('Counts by position') +
    facet_wrap(~(as.character(rep))) +
    theme_tim_label()

ggsave(plot = pCountsByPos, file = 'plots/CountsByPos.pdf')

`summarise()` regrouping output by 'var_codon_aa', 'rep' (override with `.groups` argument)



# A tibble: 4,364 x 4
   var_codon_aa   rep   pos sum_input
   <chr>        <dbl> <dbl>     <dbl>
 1 A_GCC            1   538       160
 2 A_GCC            1   539        88
 3 A_GCC            1   540       114
 4 A_GCC            1   541       118
 5 A_GCC            1   542       202
 6 A_GCC            1   543        21
 7 A_GCC            1   544        11
 8 A_GCC            1   545        31
 9 A_GCC            1   546        83
10 A_GCC            1   547       236
# … with 4,354 more rows


Saving 7 x 7 in image



In [57]:
#Enrichment histograms, by type
mNorm %>% filter(is.na(wt_variant)) %>%
    print(n = Inf, width = Inf)
pEnrichmentHistograms <- ggplot(mNorm, aes(x = sort_norm, y = ..count.., color = wt_variant)) + 
    stat_bin(geom = "step", bins = 100, position = 'identity', size = 1) +
    scale_y_continuous(expand = c(0,0)) +
    scale_x_continuous() +
    facet_wrap(~as.character(rep)) +
    theme_tim_presentation()
ggsave(plot = pEnrichmentHistograms, file = 'plots/EnrichmentHistograms.pdf', width = 8, height = 6)


# A tibble: 0 x 13
# … with 13 variables: seq_name <chr>, rep <dbl>, input <dbl>, sort <dbl>,
#   sort_norm <dbl>, wt_codon <chr>, aapos <chr>, pos <dbl>,
#   var_codon_aa <chr>, var_codon <chr>, wt_aa <chr>, var_aa <chr>,
#   wt_variant <lgl>


Warning message:
“Removed 801 rows containing non-finite values (stat_bin).”


In [60]:
#comparison plots, by codon
CompPlotsCodon <- mNorm %>% 
    filter(!wt_variant) %>%
    select(rep, sort_norm, seq_name) %>%
    pivot_wider(names_from = rep, values_from = sort_norm) %>%
    select(-seq_name)

pCompPlotsCodon <- plot_corr_grid(CompPlotsCodon)
ggsave(pCompPlotsCodon, file = 'plots/pCompPlotsCodon.pdf', width = 8, height = 8)

Warning message:
“Removed 341 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 341 rows containing missing values (geom_point).”
Warning message:
“Removed 264 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 264 rows containing missing values (geom_point).”
Warning message:
“Removed 380 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 380 rows containing missing values (geom_point).”
Warning message:
“Removed 256 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 256 rows containing missing values (geom_point).”
Warning message:
“Removed 382 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 382 rows containing missing values (geom_point).”
Warning message:
“Removed 302 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 302 rows containing missing values (geom_point).”


In [73]:
#comparison plots, succint
CompPlotsAveraged <- mNorm %>% 
    filter(!wt_variant) %>%
    select(rep, sort_norm, var_aa, pos) %>%
    group_by(rep, var_aa, pos) %>%
    summarise(aa_mean = mean(sort_norm)) %>%
    ungroup() %>%
    pivot_wider(names_from = rep, values_from = aa_mean, id_cols = c(pos, var_aa)) %>%
    print() %>%
    select(-pos, -var_aa)

pCompPlotsAveraged <- plot_corr_grid(CompPlotsAveraged)
ggsave(pCompPlotsAveraged, file = 'plots/pCompAll_CodonAveraged.pdf', width = 8, height = 8)

`summarise()` regrouping output by 'rep', 'var_aa' (override with `.groups` argument)



# A tibble: 560 x 6
     pos var_aa     `1`     `2`      `3`     `4`
   <dbl> <chr>    <dbl>   <dbl>    <dbl>   <dbl>
 1   538 A      -0.114   0.107  -0.00897 -0.0300
 2   539 A      -0.409  NA      -0.480   NA     
 3   540 A      -0.720  NA      -0.412   -0.154 
 4   541 A       0.0316 -0.257  NA       NA     
 5   542 A      -0.0322  0.422   0.262   NA     
 6   543 A      NA      -0.0365 NA       NA     
 7   544 A      NA      NA      -0.488   -0.390 
 8   545 A      NA       0.182   0.575   NA     
 9   546 A      -0.514  NA      -0.373   -0.519 
10   547 A      -0.0703 NA      -0.207   NA     
# … with 550 more rows


Warning message:
“Removed 260 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 260 rows containing missing values (geom_point).”
Warning message:
“Removed 214 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 214 rows containing missing values (geom_point).”
Warning message:
“Removed 296 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 296 rows containing missing values (geom_point).”
Warning message:
“Removed 205 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 205 rows containing missing values (geom_point).”
Warning message:
“Removed 286 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 286 rows containing missing values (geom_point).”
Warning message:
“Removed 246 rows containing non-finite values (stat_cor).”
Warning message:
“Removed 246 rows containing missing values (geom_point).”


In [91]:
#barplots, show each residue with variance. 
#precursor to heatmap, which is in the next cell.
AllDataAveragedNormalized <- mNorm %>% 
    filter(!str_detect(seq_name, '_WT_')) %>%
    group_by(pos, var_codon, aapos, var_aa, wt_variant) %>%
    summarise(avg_F = mean(sort_norm, na.rm = TRUE), #averaging over the reps.
          data_SD = sd(sort_norm, na.rm = TRUE)) %>%
    ungroup() %>%
    group_by(pos, var_aa, aapos, wt_variant) %>%
    summarise(avg_F_prop = mean(avg_F, na.rm = TRUE),
    data_SD_prop = sqrt(sum((data_SD)^2, na.rm = TRUE) / 4)) %>%
    ungroup() %>%
    distinct_all() %>%
    print()

# deal with the tryptophan issue
AllDataAveragedNormalized <- bind_rows(AllDataAveragedNormalized, tibble(
    pos = 563,
    wt_aa = 'W',
    var_aa = 'W',
    data_SD_prop = NA,
    avg_F_prop = filter(AllDataAveragedNormalized, wt_aa == 'wt')$avg_F_prop,
    aapos = 'W_563',
    wt_variant = TRUE)) 

#specify no data
AllDataAveragedNormalized <- AllDataAveragedNormalized %>% 
    mutate(BarLabels = ifelse(is.na(avg_F_prop), 'N.D.', ''))

#order rows and columns
AllDataAveragedNormalized$var_aa <- factor(AllDataAveragedNormalized$var_aa, levels = theme_roworder)
AllDataAveragedNormalized$aapos <- factor(AllDataAveragedNormalized$aapos, levels = unique(AllDataAveragedNormalized$aapos))

#plot
pAvgNorm <- ggplot(AllDataAveragedNormalized %>% filter(aapos != 'wt_0'), aes(x = var_aa, y = avg_F_prop)) +
        geom_col(aes(fill = wt_variant)) +
        geom_errorbar(aes(ymin = avg_F_prop - data_SD_prop, ymax = avg_F_prop + data_SD_prop), width = 0.5) +
        geom_text(aes(label = BarLabels, y = 0), size = 1, angle = 90) +
        facet_wrap(~aapos, ncol = 6, scales = 'free_x') +
        scale_x_discrete(name = 'Mutation') +
        scale_fill_manual(values = c('grey55', 'blue')) +
        scale_y_continuous(name = 'Mean of four replicates, with propagated error') +
        theme_tim_label() +
        theme(legend.position = "none")


ggsave(plot = pAvgNorm, file = 'plots/pAvgNorm.pdf', width = 10, height = 8)

`summarise()` regrouping output by 'pos', 'var_codon', 'aapos', 'var_aa' (override with `.groups` argument)

`summarise()` regrouping output by 'pos', 'var_aa', 'aapos' (override with `.groups` argument)



# A tibble: 587 x 6
     pos var_aa aapos wt_variant avg_F_prop data_SD_prop
   <dbl> <chr>  <chr> <lgl>           <dbl>        <dbl>
 1   538 A      S_538 FALSE         -0.0116       0.0979
 2   538 C      S_538 FALSE          0.166        0.114 
 3   538 D      S_538 FALSE         -0.454        0.219 
 4   538 E      S_538 FALSE         -0.264        0.136 
 5   538 F      S_538 FALSE          0.0551       0.182 
 6   538 G      S_538 FALSE          0.169        0.192 
 7   538 H      S_538 FALSE         -0.147        0.0992
 8   538 I      S_538 FALSE        NaN            0     
 9   538 K      S_538 FALSE         -0.786        0.191 
10   538 L      S_538 FALSE         -0.274        0.227 
# … with 577 more rows


Warning message:
“Removed 2 rows containing missing values (position_stack).”


In [97]:
#This is the heatmap cell. 
HeatmapPlotData <- AllDataAveragedNormalized %>% 
    mutate(var_aa = factor(var_aa, levels = rev(theme_roworder))) %>%
    print()

pHeatmapActLoop <- ggplot(HeatmapPlotData, aes(x = aapos, y = var_aa, fill = avg_F_prop)) +
	geom_tile() +
    geom_tile(data = HeatmapPlotData %>% filter(wt_variant), color = "black") +
    ggtitle("Activation Loop Heatmap") + 
	scale_fill_gradient2(low = "blue", mid = "white", high = "red") +
    scale_x_discrete(name = "Position") + 
    scale_y_discrete(name = "Variant") + 
    theme_tim_label() +
	theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, family = "Courier"))
    ggsave(plot = pHeatmapActLoop, file = paste0('plots/pHeatmapActLoop.pdf'), width = 6, height = 3)


# A tibble: 587 x 8
     pos var_aa aapos wt_variant avg_F_prop data_SD_prop wt_aa BarLabels
   <dbl> <fct>  <fct> <lgl>           <dbl>        <dbl> <chr> <chr>    
 1   538 A      S_538 FALSE         -0.0116       0.0979 NA    ""       
 2   538 C      S_538 FALSE          0.166        0.114  NA    ""       
 3   538 D      S_538 FALSE         -0.454        0.219  NA    ""       
 4   538 E      S_538 FALSE         -0.264        0.136  NA    ""       
 5   538 F      S_538 FALSE          0.0551       0.182  NA    ""       
 6   538 G      S_538 FALSE          0.169        0.192  NA    ""       
 7   538 H      S_538 FALSE         -0.147        0.0992 NA    ""       
 8   538 I      S_538 FALSE        NaN            0      NA    "N.D."   
 9   538 K      S_538 FALSE         -0.786        0.191  NA    ""       
10   538 L      S_538 FALSE         -0.274        0.227  NA    ""       
# … with 577 more rows
